In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the website
url = "https://halalstock.in/halal-shariah-compliant-shares-list/"

# Define headers (modify these according to the website's requirements)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.5",
}

# Send an HTTP GET request with headers
response = requests.get(url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table containing the data
    table = soup.find("table")

    # Extract column headers
    headers = [th.text.strip() for th in table.select("thead th")]

    # Use pandas to read the HTML table into a DataFrame
    df = pd.read_html(str(table))[0]

    # Rename columns with extracted headers
    df.columns = headers

    # Process the first column based on the 'src' attribute of the image tags
    df['Halal'] = [img['src'] for img in table.select("td.column-1 img")]

    # Filter rows based on the 'Halal' column containing 'hs-yes.jpg'
    filtered_df = df[df['Halal'].str.contains('hs-yes.jpg', na=False)]

    # Exclude the last column before exporting to Excel
    filtered_df = filtered_df.iloc[:, 1:-1]

    # Export the filtered DataFrame to an Excel file
    filtered_df.to_excel("halal_shares_list_filtered.xlsx", index=False)

    print("Filtered data exported to Excel successfully, excluding first and last column.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [44]:
def nsefetch(payload):
        try: #Try the normal way.
            output = requests.get(payload,headers=headers).json()
        except:
            try: # Follow Praty's hack as mentioned here at https://forum.unofficed.com/t/solution-equity-history-function-doesnt-work/1197
                output = requests.session().get(payload,headers=headers).json()

            except: #Mimick FrontPage Visit
                s=Session()

                # Emulate browser
                s.headers.update({"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"})
                #payload2 = "https://www.nseindia.com"
                #output2 = s.get(payload2,headers=headers).json()

                # Get the cookies from the main page (will update automatically in headers)
                s.get("https://www.nseindia.com/")
                output = s.get(payload).json()
                #with open("securities.csv", "w") as f:
                    #f.write(output)
        return output

#Getting FNO Symboles
def fnolist():
    # df = pd.read_csv("https://www1.nseindia.com/content/fo/fo_mktlots.csv")
    # return [x.strip(' ') for x in df.drop(df.index[3]).iloc[:,1].to_list()]

    positions = nsefetch('https://www.nseindia.com/api/equity-stockIndices?index=SECURITIES%20IN%20F%26O')

    nselist=['NIFTY','NIFTYIT','BANKNIFTY']

    i=0
    for x in range(i, len(positions['data'])):
        nselist=nselist+[positions['data'][x]['symbol']]

    return nselist

def nse_quote(symbol,section=""):
    #https://forum.unofficed.com/t/nsetools-get-quote-is-not-fetching-delivery-data-and-delivery-can-you-include-this-a>    symbol = nsesymbolpurify(symbol)

    if(section==""):
        #if any(x in symbol for x in fnolist()):
            #payload = nsefetch('https://www.nseindia.com/api/quote-derivative?symbol='+symbol)
        #else:
        payload = nsefetch('https://www.nseindia.com/api/quote-equity?symbol='+symbol)
        return payload

    if(section!=""):
        payload = nsefetch('https://www.nseindia.com/api/quote-equity?symbol='+symbol+'&section='+section)
        return payload

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


# URL of the website
url = "https://halalstock.in/halal-shariah-compliant-shares-list/"

# Define headers (modify these according to the website's requirements)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.5",
}

# Send an HTTP GET request with headers
response = requests.get(url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table containing the data
    table = soup.find("table")

    # Extract column headers
    headers = [th.text.strip() for th in table.select("thead th")]

    # Use pandas to read the HTML table into a DataFrame
    df = pd.read_html(str(table))[0]

    # Rename columns with extracted headers
    df.columns = headers

    # Process the first column based on the 'src' attribute of the image tags
    df['Halal'] = [img['src'] for img in table.select("td.column-1 img")]

    # Filter rows based on the 'Halal' column containing 'hs-yes.jpg'
    filtered_df = df[df['Halal'].str.contains('hs-yes.jpg', na=False)]

    # Exclude both the first and last columns before exporting to Excel
    filtered_df = filtered_df.iloc[:, 1:-1]

    # Remove rows where 'NSECode' column is empty
    filtered_df = filtered_df.dropna(subset=['NSECode'])
    
    # Sort the rows by the "Industry" column
    sorted_df = filtered_df.sort_values(by="Industry")

    # Export the sorted DataFrame to an Excel file
    sorted_df.to_excel("halal_shares_list_sorted_by_industry.xlsx", index=False)

    print("Filtered and sorted data exported to Excel successfully.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [45]:
#from requests import Session
import json
# Define the symbol and series
symbol = "GOCOLORS"

url = 'https://www.nseindia.com/api/quote-equity?symbol=' + symbol.replace(' ', '%20').replace('&','%26') + "&section=trade_info"
print(url)
nsedata = nsefetch(url)
nsedata2 = nse_quote(symbol)
#tc = data['marketDeptOrderBook']['tradeInfo']['totalMarketCap']
# totalTradedVolume = nsedata['marketDeptOrderBook']['tradeInfo']['totalTradedVolume']
# totalMarketCap = nsedata['marketDeptOrderBook']['tradeInfo']['totalMarketCap']
# deliveryToTradedQuantity = nsedata['securityWiseDP']['deliveryToTradedQuantity']
# print(totalTradedVolume, totalMarketCap, deliveryToTradedQuantity)
#print(nsedata)
print(nsedata2)


https://www.nseindia.com/api/quote-equity?symbol=IDEAFORGE&section=trade_info
{'info': {'symbol': 'IDEAFORGE', 'companyName': 'Ideaforge Technology Limited', 'activeSeries': ['EQ'], 'debtSeries': [], 'tempSuspendedSeries': [], 'isFNOSec': False, 'isCASec': False, 'isSLBSec': False, 'isDebtSec': False, 'isSuspended': False, 'isETFSec': False, 'isDelisted': False, 'isin': 'INE349Y01013', 'isTop10': False, 'identifier': 'IDEAFORGEEQN'}, 'metadata': {'series': 'EQ', 'symbol': 'IDEAFORGE', 'isin': 'INE349Y01013', 'status': 'Listed', 'listingDate': '07-Jul-2023', 'industry': 'Aerospace & Defense', 'lastUpdateTime': '24-Nov-2023 16:00:00', 'pdSectorPe': 'NA', 'pdSymbolPe': 'NA', 'pdSectorInd': 'NA'}, 'securityInfo': {'boardStatus': 'Main', 'tradingStatus': 'Active', 'tradingSegment': 'Normal Market', 'sessionNo': '-', 'slb': 'No', 'classOfShare': 'Equity', 'derivatives': 'No', 'surveillance': {'surv': None, 'desc': None}, 'faceValue': 10, 'issuedSize': 42215323}, 'sddDetails': {'SDDAuditor': 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Create a persistent session for halalstocks.in
session_halalstocks = requests.Session()
session_halalstocks.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.5",
})

# Function to fetch NSE data
def nsefetch(symbol):
    try:
        # Create a new session for NSE data
        session_nse = requests.Session()
        session_nse.headers.update({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.5",
        })

        # Define the NSE data URL
        url = f'https://www.nseindia.com/api/quote-equity?symbol={symbol.replace(" ", "%20").replace("&", "%26")}&section=trade_info'

                        
        session_nse.get("https://www.nseindia.com/")
        # Fetch NSE data
        response = session_nse.get(url)
        response.raise_for_status()
        data = response.json()

        # Close the session for NSE data
        session_nse.close()

        return data
    except requests.exceptions.HTTPError as errh:
        print("HTTP Error:", errh)
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:", errc)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
    except requests.exceptions.RequestException as err:
        print("Error:", err)
        print("Response content:", response.text)  # Print the response content for further inspection

# URL of the website
url = "https://halalstock.in/halal-shariah-compliant-shares-list/"

# Send an HTTP GET request with headers
response = session_halalstocks.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table containing the data
    table = soup.find("table")

    # Extract column headers
    headers = [th.text.strip() for th in table.select("thead th")]

    # Use pandas to read the HTML table into a DataFrame
    df = pd.read_html(str(table))[0]

    # Rename columns with extracted headers
    df.columns = headers

    # Process the first column based on the 'src' attribute of the image tags
    df['Halal'] = [img['src'] for img in table.select("td.column-1 img")]

    # Filter rows based on the 'Halal' column containing 'hs-yes.jpg'
    filtered_df = df[df['Halal'].str.contains('hs-yes.jpg', na=False)]

    # Exclude both the first and last columns before exporting to Excel
    filtered_df = filtered_df.iloc[:, 1:-1]

    # Remove rows where 'NSECode' column is empty
    filtered_df = filtered_df.dropna(subset=['NSECode'])

    # Sort the rows by the "Industry" column
    sorted_df = filtered_df.sort_values(by="Industry")

    # Iterate over rows and fetch NSE data
    for index, row in sorted_df.iterrows():
        symbol = row['NSECode']
        print(symbol)
        nsedata = nsefetch(symbol)

        # Extract required data from NSE response
        try:
            totalTradedVolume = nsedata['marketDeptOrderBook']['tradeInfo']['totalTradedVolume']
        except KeyError:
            totalTradedVolume = 0

        try:
            totalMarketCap = nsedata['marketDeptOrderBook']['tradeInfo']['totalMarketCap']
        except KeyError:
            totalMarketCap = 0

        try:
            deliveryToTradedQuantity = nsedata['securityWiseDP']['deliveryToTradedQuantity']
        except KeyError:
            deliveryToTradedQuantity = 0
        # Print or store the fetched data as needed
        print(f"NSECode: {symbol}, Total Traded Volume: {totalTradedVolume}, Total Market Cap: {totalMarketCap}, Delivery to Traded Quantity: {deliveryToTradedQuantity}")

    # Export the sorted DataFrame to an Excel file
    sorted_df.to_excel("halal_shares_list_sorted.xlsx", index=False)

    print("Filtered and sorted data exported to Excel successfully.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Create a persistent session for halalstocks.in
session_halalstocks = requests.Session()
session_halalstocks.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.5",
})

# URL of the website
url = "https://halalstock.in/halal-shariah-compliant-shares-list/"

# Send an HTTP GET request with headers
response = session_halalstocks.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table containing the data
    table = soup.find("table")

    # Extract column headers
    headers = [th.text.strip() for th in table.select("thead th")]

    # Use pandas to read the HTML table into a DataFrame
    df = pd.read_html(str(table))[0]

    # Rename columns with extracted headers
    df.columns = headers

    # Process the first column based on the 'src' attribute of the image tags
    df['Halal'] = [img['src'] for img in table.select("td.column-1 img")]

    # Filter rows based on the 'Halal' column containing 'hs-yes.jpg'
    filtered_df = df[df['Halal'].str.contains('hs-yes.jpg', na=False)]

    # Exclude both the first and last columns before exporting to Excel
    filtered_df = filtered_df.iloc[:, 1:-1]

    # Remove rows where 'NSECode' column is empty
    filtered_df = filtered_df.dropna(subset=['NSECode'])
    
    # Sort the rows by the "Industry" column
    sorted_df = filtered_df.sort_values(by="Industry")

    # Export the sorted DataFrame to an Excel file
    sorted_df.to_csv("halal_shares_list_sorted_by_industry.csv", index=False)

    print("Filtered and sorted data exported to Excel successfully.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
# Close the session for halalstocks.in
session_halalstocks.close()


In [1]:
import requests
from requests import Session
import pandas as pd

# Function to fetch NSE data
def nsefetch(symbol):
    try:
        # Create a new session for NSE data
        session_nse = requests.Session()
        session_nse.headers.update({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.5",
        })

        # Define the NSE data URL
        url = f'https://www.nseindia.com/api/quote-equity?symbol={symbol.replace(" ", "%20").replace("&", "%26")}&section=trade_info'

        session_nse.get("https://www.nseindia.com/")
        # Fetch NSE data
        response = session_nse.get(url)
        response.raise_for_status()
        data = response.json()

        # Close the session for NSE data
        session_nse.close()

        return data
    except requests.exceptions.HTTPError as errh:
        print("HTTP Error:", errh)
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:", errc)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
    except requests.exceptions.RequestException as err:
        print("Error:", err)
        print("Response content:", response.text)  # Print the response content for further inspection

# Load the Excel file with NSECode column
excel_file_path = "nse_halal.csv"
#df = pd.read_excel(excel_file_path)
df = pd.read_csv(excel_file_path)
# Iterate over rows and fetch NSE data
for index, row in df.iterrows():
    symbol = row['NSECode']
    print(symbol)
    nsedata = nsefetch(symbol)

    # Extract required data from NSE response or set to zero if not present
    try:
        totalTradedVolume = nsedata['marketDeptOrderBook']['tradeInfo']['totalTradedVolume']
    except KeyError:
        totalTradedVolume = 0

    try:
        totalMarketCap = nsedata['marketDeptOrderBook']['tradeInfo']['totalMarketCap']
    except KeyError:
        totalMarketCap = 0

    try:
        deliveryToTradedQuantity = nsedata['securityWiseDP']['deliveryToTradedQuantity']
    except KeyError:
        deliveryToTradedQuantity = 0
    
    print(f"row: {index}, NSECode: {symbol}, Total Traded Volume: {totalTradedVolume}, Total Market Cap: {totalMarketCap}, Delivery to Traded Quantity: {deliveryToTradedQuantity}")

    # Update the DataFrame with new columns
    df.at[index, 'TotalTradedVolume'] = totalTradedVolume
    df.at[index, 'TotalMarketCap'] = totalMarketCap
    df.at[index, 'DeliveryToTradedQuantity'] = deliveryToTradedQuantity

# Save the updated DataFrame to the same Excel file
df.to_csv(excel_file_path, index=False)

print("Updated data exported to Excel successfully.")


SANSERA
row: 0, NSECode: SANSERA, Total Traded Volume: 92050, Total Market Cap: 452023.45, Delivery to Traded Quantity: 57.16
GRINDWELL
row: 1, NSECode: GRINDWELL, Total Traded Volume: 29552, Total Market Cap: 2336192, Delivery to Traded Quantity: 74.81
WENDT
row: 2, NSECode: WENDT, Total Traded Volume: 551, Total Market Cap: 283700, Delivery to Traded Quantity: 52.63
CARBORUNIV
row: 3, NSECode: CARBORUNIV, Total Traded Volume: 305792, Total Market Cap: 2152660.02, Delivery to Traded Quantity: 72.23
DCXINDIA
row: 4, NSECode: DCXINDIA, Total Traded Volume: 1783225, Total Market Cap: 355798.02, Delivery to Traded Quantity: 53.33
IDEAFORGE
row: 5, NSECode: IDEAFORGE, Total Traded Volume: 77050, Total Market Cap: 326662.17, Delivery to Traded Quantity: 56.87
JCHAC
row: 6, NSECode: JCHAC, Total Traded Volume: 32815, Total Market Cap: 326290.61, Delivery to Traded Quantity: 44.69
BLUESTARCO
row: 7, NSECode: BLUESTARCO, Total Traded Volume: 349192, Total Market Cap: 2082672.19, Delivery to Tr

row: 64, NSECode: ABMINTLLTD, Total Traded Volume: 589, Total Market Cap: 4327.68, Delivery to Traded Quantity: 0
SOMICONVEY
row: 65, NSECode: SOMICONVEY, Total Traded Volume: 16011, Total Market Cap: 10837.28, Delivery to Traded Quantity: 0
SPLPETRO
row: 66, NSECode: SPLPETRO, Total Traded Volume: 58136, Total Market Cap: 1053971.72, Delivery to Traded Quantity: 59.19
ANTGRAPHIC
row: 67, NSECode: ANTGRAPHIC, Total Traded Volume: 431583, Total Market Cap: 1627.6, Delivery to Traded Quantity: 100
AETHER
row: 68, NSECode: AETHER, Total Traded Volume: 40526, Total Market Cap: 1177046.42, Delivery to Traded Quantity: 54.02
MENONBE
row: 69, NSECode: MENONBE, Total Traded Volume: 42087, Total Market Cap: 76634.7, Delivery to Traded Quantity: 60.53
NRBBEARING
row: 70, NSECode: NRBBEARING, Total Traded Volume: 138562, Total Market Cap: 250593.38, Delivery to Traded Quantity: 40.19
SKFINDIA
row: 71, NSECode: SKFINDIA, Total Traded Volume: 14304, Total Market Cap: 2272663.16, Delivery to Traded 

row: 128, NSECode: CHEMFAB, Total Traded Volume: 16373, Total Market Cap: 52562.21, Delivery to Traded Quantity: 82.49
CHEMPLASTS
row: 129, NSECode: CHEMPLASTS, Total Traded Volume: 56719, Total Market Cap: 705089.65, Delivery to Traded Quantity: 36.56
CLEAN
row: 130, NSECode: CLEAN, Total Traded Volume: 60537, Total Market Cap: 1433278.03, Delivery to Traded Quantity: 70.63
SOLARINDS
row: 131, NSECode: SOLARINDS, Total Traded Volume: 112024, Total Market Cap: 6130701.23, Delivery to Traded Quantity: 56.77
SRHHYPOLTD
row: 132, NSECode: SRHHYPOLTD, Total Traded Volume: 13455, Total Market Cap: 93651.26, Delivery to Traded Quantity: 56.48
CHEMBOND
row: 133, NSECode: CHEMBOND, Total Traded Volume: 83456, Total Market Cap: 68317.3, Delivery to Traded Quantity: 31.27
OAL
row: 134, NSECode: OAL, Total Traded Volume: 6319, Total Market Cap: 116491.85, Delivery to Traded Quantity: 44.52
EXCELINDUS
row: 135, NSECode: EXCELINDUS, Total Traded Volume: 6783, Total Market Cap: 105317.26, Delivery t

row: 192, NSECode: TVSELECT, Total Traded Volume: 31531, Total Market Cap: 64334.27, Delivery to Traded Quantity: 37.09
CEREBRAINT
row: 193, NSECode: CEREBRAINT, Total Traded Volume: 93599, Total Market Cap: 7783.06, Delivery to Traded Quantity: 0
GVPTECH
row: 194, NSECode: GVPTECH, Total Traded Volume: 48963, Total Market Cap: 17100.47, Delivery to Traded Quantity: 0
LCCINFOTEC
row: 195, NSECode: LCCINFOTEC, Total Traded Volume: 105830, Total Market Cap: 2468.57, Delivery to Traded Quantity: 72.38
LTIM
row: 196, NSECode: LTIM, Total Traded Volume: 152012, Total Market Cap: 16174903.56, Delivery to Traded Quantity: 31.71
WIPRO
row: 197, NSECode: WIPRO, Total Traded Volume: 3125283, Total Market Cap: 20680203.88, Delivery to Traded Quantity: 46.09
INFY
row: 198, NSECode: INFY, Total Traded Volume: 2973195, Total Market Cap: 59641197.32, Delivery to Traded Quantity: 61.71
TCS
row: 199, NSECode: TCS, Total Traded Volume: 1989733, Total Market Cap: 126383634.42, Delivery to Traded Quantity

row: 257, NSECode: ADROITINFO, Total Traded Volume: 7166, Total Market Cap: 4154.81, Delivery to Traded Quantity: 0
BCG
row: 258, NSECode: BCG, Total Traded Volume: 5476159, Total Market Cap: 339010.87, Delivery to Traded Quantity: 59.41
HAPPSTMNDS
row: 259, NSECode: HAPPSTMNDS, Total Traded Volume: 129506, Total Market Cap: 1269971.92, Delivery to Traded Quantity: 55.36
RAMCOSYS
row: 260, NSECode: RAMCOSYS, Total Traded Volume: 89167, Total Market Cap: 98498.15, Delivery to Traded Quantity: 60.03
SIGMA
row: 261, NSECode: SIGMA, Total Traded Volume: 18009, Total Market Cap: 47276.49, Delivery to Traded Quantity: 43.77
SAKSOFT
row: 262, NSECode: SAKSOFT, Total Traded Volume: 31969, Total Market Cap: 371090.48, Delivery to Traded Quantity: 0
63MOONS
row: 263, NSECode: 63MOONS, Total Traded Volume: 841791, Total Market Cap: 190350.44, Delivery to Traded Quantity: 51.24
SILVERTUC
row: 264, NSECode: SILVERTUC, Total Traded Volume: 7018, Total Market Cap: 82153.86, Delivery to Traded Quantit

...

row: 708, NSECode: SEQUENT, Total Traded Volume: 12978532, Total Market Cap: 305930.18, Delivery to Traded Quantity: 24.59
NEULANDLAB
row: 709, NSECode: NEULANDLAB, Total Traded Volume: 12598, Total Market Cap: 650475.37, Delivery to Traded Quantity: 48.4
AARTIDRUGS
row: 710, NSECode: AARTIDRUGS, Total Traded Volume: 130330, Total Market Cap: 441104.13, Delivery to Traded Quantity: 34.38
DIVISLAB
row: 711, NSECode: DIVISLAB, Total Traded Volume: 829615, Total Market Cap: 9981618.61, Delivery to Traded Quantity: 38.9
MARKSANS
row: 712, NSECode: MARKSANS, Total Traded Volume: 2014210, Total Market Cap: 643492.52, Delivery to Traded Quantity: 38.64
GRANULES
row: 713, NSECode: GRANULES, Total Traded Volume: 7834685, Total Market Cap: 935543.24, Delivery to Traded Quantity: 29.32
LASA
row: 714, NSECode: LASA, Total Traded Volume: 64771, Total Market Cap: 13076.42, Delivery to Traded Quantity: 49.36
ANURAS
row: 715, NSECode: ANURAS, Total Traded Volume: 284599, Total Market Cap: 1024453.75, 

row: 772, NSECode: ASTRAZEN, Total Traded Volume: 2636, Total Market Cap: 1170000, Delivery to Traded Quantity: 61.19
UNIVPHOTO
row: 773, NSECode: UNIVPHOTO, Total Traded Volume: 1530, Total Market Cap: 44902.97, Delivery to Traded Quantity: 65.42
TEXMOPIPES
row: 774, NSECode: TEXMOPIPES, Total Traded Volume: 246789, Total Market Cap: 21326.95, Delivery to Traded Quantity: 61.11
ASTRAL
row: 775, NSECode: ASTRAL, Total Traded Volume: 184033, Total Market Cap: 5192560.08, Delivery to Traded Quantity: 31.47
PREMIERPOL
row: 776, NSECode: PREMIERPOL, Total Traded Volume: 19909, Total Market Cap: 32742.5, Delivery to Traded Quantity: 0
XPROINDIA
row: 777, NSECode: XPROINDIA, Total Traded Volume: 33259, Total Market Cap: 191424.86, Delivery to Traded Quantity: 50.15
RESPONIND
row: 778, NSECode: RESPONIND, Total Traded Volume: 54668, Total Market Cap: 868400.47, Delivery to Traded Quantity: 55.86
FINPIPE
row: 779, NSECode: FINPIPE, Total Traded Volume: 647078, Total Market Cap: 1240953.81, Del

row: 836, NSECode: MANYAVAR, Total Traded Volume: 285045, Total Market Cap: 3238960.54, Delivery to Traded Quantity: 65.11
SURYAROSNI
row: 837, NSECode: SURYAROSNI, Total Traded Volume: 174756, Total Market Cap: 548442.46, Delivery to Traded Quantity: 44.49
APLAPOLLO
row: 838, NSECode: APLAPOLLO, Total Traded Volume: 1090186, Total Market Cap: 4695210.68, Delivery to Traded Quantity: 49.99
GANDHITUBE
row: 839, NSECode: GANDHITUBE, Total Traded Volume: 4505, Total Market Cap: 87459.03, Delivery to Traded Quantity: 77.47
KSL
row: 840, NSECode: KSL, Total Traded Volume: 39853, Total Market Cap: 208006.83, Delivery to Traded Quantity: 49.61
RATNAMANI
row: 841, NSECode: RATNAMANI, Total Traded Volume: 38559, Total Market Cap: 2460229.2, Delivery to Traded Quantity: 40.8
TIIL
row: 842, NSECode: TIIL, Total Traded Volume: 7653, Total Market Cap: 498268.61, Delivery to Traded Quantity: 62.09
RAJRATAN
row: 843, NSECode: RAJRATAN, Total Traded Volume: 12144, Total Market Cap: 376593.89, Delivery

row: 900, NSECode: WEIZMANIND, Total Traded Volume: 10780, Total Market Cap: 18911.88, Delivery to Traded Quantity: 0
JAIPURKURT
row: 901, NSECode: JAIPURKURT, Total Traded Volume: 182975, Total Market Cap: 9084.27, Delivery to Traded Quantity: 63.76
MITTAL
row: 902, NSECode: MITTAL, Total Traded Volume: 2445716, Total Market Cap: 5178.84, Delivery to Traded Quantity: 73.87
TRIDENT
row: 903, NSECode: TRIDENT, Total Traded Volume: 4268264, Total Market Cap: 1880407.64, Delivery to Traded Quantity: 46.56
JINDWORLD
row: 904, NSECode: JINDWORLD, Total Traded Volume: 45836, Total Market Cap: 661717.32, Delivery to Traded Quantity: 31.64
IRISDOREME
row: 905, NSECode: IRISDOREME, Total Traded Volume: 118798, Total Market Cap: 65256.5, Delivery to Traded Quantity: 43.78
RUPA
row: 906, NSECode: RUPA, Total Traded Volume: 648328, Total Market Cap: 217102.05, Delivery to Traded Quantity: 29.04
GOKEX
row: 907, NSECode: GOKEX, Total Traded Volume: 294950, Total Market Cap: 579221.77, Delivery to Tr

row: 965, NSECode: IRCTC, Total Traded Volume: 896138, Total Market Cap: 5541600, Delivery to Traded Quantity: 38.5
THOMASCOOK
row: 966, NSECode: THOMASCOOK, Total Traded Volume: 452488, Total Market Cap: 746964.33, Delivery to Traded Quantity: 53.35
BALKRISIND
row: 967, NSECode: BALKRISIND, Total Traded Volume: 294957, Total Market Cap: 4964385.44, Delivery to Traded Quantity: 66.81
MRF
row: 968, NSECode: MRF, Total Traded Volume: 2854, Total Market Cap: 4721038.93, Delivery to Traded Quantity: 29.85
SUNPHARMA
row: 969, NSECode: SUNPHARMA, Total Traded Volume: 1376695, Total Market Cap: 28649259.21, Delivery to Traded Quantity: 48.11
Updated data exported to Excel successfully.


In [ ]:
import pandas as pd

# Read the two sheets
df1 = pd.read_csv("halal_shares_list_sorted_by_industry.csv", delimiter=',')
df2 = pd.read_csv("EQUITY_L.csv", delimiter=',')

# Merge the dataframes based on the first column
merged_df = pd.merge(df1, df2, left_on='NSECode', right_on='SYMBOL', how='inner')

# Save the result to sheet3.csv
merged_df.to_csv("path_to_sheet3.csv", index=False, sep=',')
